In [2]:
#Import packages
import numpy as np
import scipy as sp
import matplotlib as ml
import os
import sys
flopypth = os.path.join('..', '..', 'flopy')
if flopypth not in sys.path:
    sys.path.append(flopypth)
import flopy
import flopy.utils.binaryfile as bf
import subprocess
import matplotlib.pyplot as plt
import shutil
from scipy.stats import uniform

In [114]:
hk_min = 1e-2
hk_max = 10
vka_min = 1e-3
vka_max = 1
numWells = 2
pump_min = -50000.
pump_max = -3000.
sampleSize = 10
n = 2

In [13]:
# Generate arrays of hk and vka
hk = np.linspace(hk_min, hk_max, num=sampleSize)
vka = np.linspace(vka_min,vka_max,num=sampleSize)
hk
vka

array([ 0.001,  0.112,  0.223,  0.334,  0.445,  0.556,  0.667,  0.778,
        0.889,  1.   ])

In [8]:
list(range(numWells))

[0, 1]

In [18]:
params_in_sample = ['hk', 'vka']
for n in range(numWells):
    params_in_sample.append('pump_rate_' + str(n+1))
params_in_sample

['hk', 'vka', 'pump_rate_1', 'pump_rate_2']

In [69]:
pump = np.zeros([numWells, sampleSize])
for n in range(numWells):
    pump[n,:] = np.linspace(pump_min, pump_max, num=sampleSize)
        
pumpSplit

[array([[  3000.        ,   8222.22222222,  13444.44444444,  18666.66666667,
          23888.88888889,  29111.11111111,  34333.33333333,  39555.55555556,
          44777.77777778,  50000.        ]]),
 array([[  3000.        ,   8222.22222222,  13444.44444444,  18666.66666667,
          23888.88888889,  29111.11111111,  34333.33333333,  39555.55555556,
          44777.77777778,  50000.        ]])]

In [71]:
pumpSplit = np.split(pump,numWells)
param_sample = np.stack([hk, vka])
for i in range(numWells):
    param_sample = np.append(param_sample,pumpSplit[i], axis=0)
param_sample

array([[  1.00000000e-02,   1.12000000e+00,   2.23000000e+00,
          3.34000000e+00,   4.45000000e+00,   5.56000000e+00,
          6.67000000e+00,   7.78000000e+00,   8.89000000e+00,
          1.00000000e+01],
       [  1.00000000e-03,   1.12000000e-01,   2.23000000e-01,
          3.34000000e-01,   4.45000000e-01,   5.56000000e-01,
          6.67000000e-01,   7.78000000e-01,   8.89000000e-01,
          1.00000000e+00],
       [  3.00000000e+03,   8.22222222e+03,   1.34444444e+04,
          1.86666667e+04,   2.38888889e+04,   2.91111111e+04,
          3.43333333e+04,   3.95555556e+04,   4.47777778e+04,
          5.00000000e+04],
       [  3.00000000e+03,   8.22222222e+03,   1.34444444e+04,
          1.86666667e+04,   2.38888889e+04,   2.91111111e+04,
          3.43333333e+04,   3.95555556e+04,   4.47777778e+04,
          5.00000000e+04]])

In [73]:
# Generate list of all paramters
params_in_sample = ['hk', 'vka']
for n in range(numWells):
    params_in_sample.append('pump_rate_' + str(n+1))

# Generate arrays of hk and vka
hk = np.linspace(hk_min, hk_max, num=sampleSize)
vka = np.linspace(vka_min,vka_max,num=sampleSize)

# Generate arrays of pumping rate
pump = np.zeros([numWells, sampleSize])
for n in range(numWells):
    pump[n,:] = np.linspace(pump_min, pump_max, num=sampleSize)
    
# Combine to form paramSample
pumpSplit = np.split(pump,numWells)
param_sample = np.stack([hk, vka])
for i in range(numWells):
    param_sample = np.append(param_sample,pumpSplit[i], axis=0)
    
param_sample

array([[  1.00000000e-02,   1.12000000e+00,   2.23000000e+00,
          3.34000000e+00,   4.45000000e+00,   5.56000000e+00,
          6.67000000e+00,   7.78000000e+00,   8.89000000e+00,
          1.00000000e+01],
       [  1.00000000e-03,   1.12000000e-01,   2.23000000e-01,
          3.34000000e-01,   4.45000000e-01,   5.56000000e-01,
          6.67000000e-01,   7.78000000e-01,   8.89000000e-01,
          1.00000000e+00],
       [  3.00000000e+03,   8.22222222e+03,   1.34444444e+04,
          1.86666667e+04,   2.38888889e+04,   2.91111111e+04,
          3.43333333e+04,   3.95555556e+04,   4.47777778e+04,
          5.00000000e+04],
       [  3.00000000e+03,   8.22222222e+03,   1.34444444e+04,
          1.86666667e+04,   2.38888889e+04,   2.91111111e+04,
          3.43333333e+04,   3.95555556e+04,   4.47777778e+04,
          5.00000000e+04]])

In [140]:
from scipy.stats import uniform
scale = hk_max - hk_min
hk_sample = uniform.rvs(loc=hk_min, scale=scale, size=sampleSize)
hk_sample

array([ 9.86352585,  3.0567227 ,  5.1064603 ,  2.47422919,  9.82114895,
        3.85264791,  1.67685073,  5.15602897,  1.45015831,  7.03355182])

In [143]:
# Write function to generate range of inputs for hk, vka, pumping rate in each well
def genParamSamples(hk_min, hk_max, vka_min, vka_max, numWells, pump_min, pump_max, sampleSize):
    
    # Generate list of all paramters
    params_in_sample = ['hk', 'vka']
    for n in range(numWells):
        params_in_sample.append('pump_rate_' + str(n+1))
    
    # Generate arrays of hk and vka
    scale = hk_max - hk_min
    hk = uniform.rvs(loc=hk_min, scale=scale, size=sampleSize)
    scale = vka_max - vka_min
    vka = uniform.rvs(loc=hk_min, scale=scale, size=sampleSize)
    
    # Generate arrays of pumping rate
    pump = np.zeros([numWells, sampleSize])
    scale = pump_max - pump_min
    for n in range(numWells):
        pump[n, :] = uniform.rvs(pump_min, scale, size=sampleSize)
        
    # Combine to form paramSample
    pumpSplit = np.split(pump, numWells)
    param_sample = np.stack([hk, vka])
    for i in range(numWells):
        param_sample = np.append(param_sample, pumpSplit[i], axis=0)

    # Create dictionary with samples for each parameter
    params = dict(zip(params_in_sample, param_sample))

    return params

In [144]:
params = genParamSamples(hk_min, hk_max, vka_min, vka_max, numWells, pump_min, pump_max, sampleSize)
params

{'hk': array([ 5.94855782,  5.71326747,  4.96540587,  4.99734681,  4.64978617,
         2.18217676,  5.74420255,  9.38571848,  1.25227806,  2.38145131]),
 'pump_rate_1': array([-32445.99701245, -14055.27691044, -28244.41076453, -10287.33729392,
        -30168.16719514, -27383.9781936 , -42064.94235876, -45265.03809563,
        -45442.05759994, -40977.46870541]),
 'pump_rate_2': array([ -3967.54773596, -25106.08057599, -44098.08115066,  -9046.10230226,
        -17888.46277586, -44776.90212225, -18730.32192548, -20231.62520884,
         -8098.59770566,  -3015.78723074]),
 'vka': array([ 0.71915011,  0.17298838,  0.03453191,  0.45306955,  0.11733434,
         0.22357878,  0.13651623,  0.33002774,  0.7550111 ,  0.47872082])}

In [89]:
for i in range(sampleSize):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [91]:
params.keys()

dict_keys(['hk', 'vka', 'pump_rate_1', 'pump_rate_2'])

In [113]:
pumping_rate1 = params['pump_rate_1'][9]
pumping_rate1

50000.0

In [109]:
list(params.values())[:]

[array([  0.01,   1.12,   2.23,   3.34,   4.45,   5.56,   6.67,   7.78,
          8.89,  10.  ]),
 array([ 0.001,  0.112,  0.223,  0.334,  0.445,  0.556,  0.667,  0.778,
         0.889,  1.   ]),
 array([  3000.        ,   8222.22222222,  13444.44444444,  18666.66666667,
         23888.88888889,  29111.11111111,  34333.33333333,  39555.55555556,
         44777.77777778,  50000.        ]),
 array([  3000.        ,   8222.22222222,  13444.44444444,  18666.66666667,
         23888.88888889,  29111.11111111,  34333.33333333,  39555.55555556,
         44777.77777778,  50000.        ])]

In [122]:
# Define output parameters for each run
model_name = []
i = 0
for i in range(sampleSize):
    model_name.append('model' + str(i))
model_name[i]

'model9'

In [127]:
i=0
headobj = bf.HeadFile(model_name[i]+'.hds')

FileNotFoundError: [Errno 2] No such file or directory: 'model0.hds'

In [137]:
t = [ 146.    0.]
print(t)

SyntaxError: invalid syntax (<ipython-input-137-923bfb4814c9>, line 1)

In [150]:
model_name = 'model' + str(9)
os.chdir('Stochastic_MODFLOW_repo')
os.getcwd()
os.f

'/Users/sarahfletcher/Desktop/Code/MODFLOW/Stochastic_MODFLOW_repo'

In [154]:
os.remove(model_name + '.wel')